#Inplementation

In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 1s (18.8 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u312-b07-0ubunt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Problem

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
# Create SparkSession 
spark=SparkSession.builder.getOrCreate()

"spark = SparkSession.builder       .master('local[1]')       .appName('SparkByExamples.com')       .getOrCreate() "

#Solution-1

In [ ]:
df=spark.read.csv('drive/MyDrive/HW2/2007.csv',header=True,inferSchema=True)
df=df.withColumn('ArrDelay',df['ArrDelay'].cast(IntegerType()))
df=df.withColumn('DepDelay',df['DepDelay'].cast(IntegerType()))

In [ ]:
for i in range(1,13):
  print(df.filter(df.Month==i).select(max('ArrDelay')).collect()[0],
        df.filter(df.Month==i).select(max('DepDelay')).collect()[0])

Row(max(ArrDelay)=1426) Row(max(DepDelay)=1406)
Row(max(ArrDelay)=1359) Row(max(DepDelay)=1340)
Row(max(ArrDelay)=1564) Row(max(DepDelay)=1547)
Row(max(ArrDelay)=1402) Row(max(DepDelay)=1415)
Row(max(ArrDelay)=1429) Row(max(DepDelay)=1416)
Row(max(ArrDelay)=1351) Row(max(DepDelay)=1360)
Row(max(ArrDelay)=1386) Row(max(DepDelay)=1369)
Row(max(ArrDelay)=1472) Row(max(DepDelay)=1449)
Row(max(ArrDelay)=1665) Row(max(DepDelay)=1689)
Row(max(ArrDelay)=2598) Row(max(DepDelay)=2601)
Row(max(ArrDelay)=1146) Row(max(DepDelay)=1137)
Row(max(ArrDelay)=1942) Row(max(DepDelay)=1956)


#Solution-2

In [ ]:
df_list=[]
for i in range(6):
  df_list.append(spark.read.csv('drive/MyDrive/HW2/200'+str(i)+'.csv',header=True,inferSchema=True))

In [ ]:
for i in range(6):
  print('200'+str(i)+':',df_list[i].select('SecurityDelay').distinct().count())

2000: 1
2001: 1
2002: 1
2003: 169
2004: 216
2005: 182


In [ ]:
for i in range(3,6):
  df=df_list[i]
  df=df.withColumn('SecurityDelay',df['SecurityDelay'].cast(IntegerType()))
  print('200'+str(i)+':',df.filter(df.SecurityDelay>0).count())

2003: 3740
2004: 8158
2005: 6627


#Solution-3

In [ ]:
df=spark.read.csv('drive/MyDrive/HW2/2008.csv',header=True,inferSchema=True)
df=df.withColumn('ArrDelay',df['ArrDelay'].cast(IntegerType()))
df=df.withColumn('DepDelay',df['DepDelay'].cast(IntegerType()))

In [ ]:
ArrDelay_df=df.filter(df.ArrDelay>0).groupby('Origin').count()
DepDelay_df=df.filter(df.DepDelay>0).groupby('Dest').count()

In [ ]:
ArrDelay_df=ArrDelay_df.withColumnRenamed('count','Arrcount')
DepDelay_df=DepDelay_df.withColumnRenamed('count','Depcount')

In [ ]:
Delay_df=ArrDelay_df.join(DepDelay_df,ArrDelay_df.Origin==DepDelay_df.Dest,'outer')

In [ ]:
Delay_df=Delay_df.fillna(0)

In [ ]:
Delay_df=Delay_df.withColumn('total',Delay_df.Arrcount+Delay_df.Depcount)

In [ ]:
import pandas as pd
Delay_pd=Delay_df.toPandas()

In [ ]:
Delay_pd.to_csv('Delay.csv')

In [ ]:
Delay_pd

,Origin,Arrcount,Dest,Depcount,total
0,ABE,1786,ABE,1645,3431
1,ABI,743,ABI,1108,1851
2,ABQ,15852,ABQ,19022,34874
3,ABY,524,ABY,412,936
4,ACK,235,ACK,179,414
...,...,...,...,...,...
300,WYS,23,WYS,63,86
301,XNA,5810,XNA,5288,11098
302,YAK,245,YAK,317,562
303,YKM,198,YKM,89,287


In [ ]:
print('{:15}: '.format('Origin'),df.select('Origin').distinct().count())
print('{:15}: '.format('Dest'),df.select('Dest').distinct().count())

Origin         :  303
Dest           :  304


In [ ]:
df.filter(df.Origin=='PUB').select('Dest').distinct().collect()

[Row(Origin='PUB')]